In [ ]:
import os
import logging

log_folder = 'logs'
if not os.path.exists(log_folder):
    os.makedirs(log_folder)
logging.basicConfig(filename='logs/jupyter_log.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML
logging.info('[Connection Page] Creating top buttons')
# Define the URLs for the links
url1 = "/lab/"
url2 = "/voila/render/ui/qprof_main.ipynb"

# Define the button labels
button_label1 = "Go to Home Page"
button_label2 = "Go to Query Profiler"

# Define HTML links for buttons
button_html1 = f"<a href='{url1}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px; margin-right: 10px;'><i class='fa fa-home'></i></a>"
button_html2 = f"<a href='{url2}' style='text-decoration: none; padding: 8px; background-color: #007bff; color: white; border-radius: 5px;'>{button_label2}</a>"



# Create HTML widgets
html_button1 = widgets.HTML(value=button_html1)
html_button2 = widgets.HTML(value=button_html2)

# Display buttons
display(widgets.HBox([html_button1, html_button2]))


_________

In [ ]:
# Importing libraries
logging.info('[Connection Page] Importing libraries')
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display,clear_output, HTML
from ipyfilechooser import FileChooser
import ssl

import verticapy as vp
logging.info('[Connection Page] Imported all libraries')

In [ ]:
# input text boxes
logging.info('[Connection Page] Creating the widgets for input')
host = widgets.Text(description = "Host",placeholder = "Enter Host (default is 'verticadb')", layout=widgets.Layout(width='350px'))
port = widgets.Text(description = "Port", placeholder = "Enter Port (default is 5433)", layout=widgets.Layout(width='350px'))
user = widgets.Text(description = "User name", placeholder = "Enter User Name (default is 'dbadmin')", layout=widgets.Layout(width='350px'))
password = widgets.Password(description = "Password", placeholder = "Enter Password (default is Empty)", layout=widgets.Layout(width='350px'))
database = widgets.Text(description = "Database", placeholder = "Enter Database (default is 'demo')", layout=widgets.Layout(width='350px'))
name = widgets.Text(description = "Name", placeholder = "Enter Name (default is 'VerticaDSN')", layout=widgets.Layout(width='350px'))

connection_timeout = widgets.Text(description = "Connection timeout", placeholder = "Time in seconds (default is 5)", layout=widgets.Layout(width='350px'))
connection_timeout.style.description_width = '130px'
autocommit = widgets.Checkbox(value = False, description = 'Autocommit', disabled = False, indent = False,layout=widgets.Layout(width = '350px',align_items='flex-end'))

#Oauth
oauth_access_token = widgets.Password(description = "OAuth Access Token", placeholder = "Enter Access Token", layout=widgets.Layout(width='500px'))
oauth_access_token.style.description_width = '150px'
oauth_refresh_token = widgets.Password(description = "OAuth Refresh Token", placeholder = "Enter Refresh Token", layout=widgets.Layout(width='500px'))
oauth_refresh_token.style.description_width = '150px'

#Kerberos
kerberos_host_name = widgets.Text(description = "Kerberos host name", placeholder = "Enter Host", layout=widgets.Layout(width='500px'))
kerberos_host_name.style.description_width = '150px'
kerberos_service_name = widgets.Text(description = "Kerberos service name", placeholder = "Enter Name", layout=widgets.Layout(width='500px'))
kerberos_service_name.style.description_width = '150px'

backup_server_node = ["localhost"]
connection_inputs = widgets.VBox([
    host,
    port,
    user,
    password,
    database,
    name,
    connection_timeout
])

logging.info('[Connection Page] Created all widgets')

In [ ]:
%%html
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <style>
        .header-container {
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 10px;
        }
        .header-container h1,
        .header-container a {
            margin: 0;
            margin-left: 42px;
        }
    </style>
    <title>Connect to Vertica</title>
</head>
<body>
    <div class="header-container">
        <h1>Connect to Vertica</h1>
        <a href="https://www.vertica.com/python/documentation/1.0.x/html/connection.html">Click here for more information on connection settings</a>
    </div>
    <div style="margin-left: 42px; padding:10px; font-size: 1rem">
        Choose how you connect to Vertica
    </div>
</body>
</html>

In [ ]:
# styles controls
top_box_height = "370px"
output_width = "530px"
button_color = '#007bff'
text_color = 'white'

In [ ]:
# Advanced connection widgets
dropdown = widgets.Dropdown(
    options=['', 'OAuth', 'TLS', 'Kerberos'],
    description='Advanced connection',
    disabled=False,
    layout={'width': "300px"}
)
dropdown.style.description_width = '130px'
output_advanced_connection = widgets.Output(layout={'width': output_width})

oauth_params = widgets.VBox([oauth_access_token,oauth_refresh_token])
kerberos_params = widgets.VBox([kerberos_host_name,kerberos_service_name])

# TLS widget
tls_params = widgets.RadioButtons(
    options=['Require', 'Verify-CA', 'Verify-Full'],
    value='Require', # Defaults to 'Require'
    description='Select:',
    disabled=False
)
ssl_context = None
uploader = FileChooser('')
output_tls = widgets.Output()
# Function to display appropriate widget based on radio button selection
def display_widget_tls(selection):
    output_tls.clear_output()
    global ssl_context
    with output_tls:
        if selection == 'Verify-CA' or selection == 'Verify-Full':
            display(uploader)
        if selection == "Verify-CA":
            ssl_context.check_hostname = False
        elif selection == "Verify-Full":
            ssl_context.check_hostname = True

# Call function when radio button value changes
tls_params.observe(lambda change: display_widget_tls(change['new']), names='value')

logging.info('[Connection Page] TLS widegt done')

# Function to display appropriate widget based on dropdown selection
def display_widget_advanced_connection(selection):
    global ssl_context
    output_advanced_connection.clear_output()

    with output_advanced_connection:
        if selection == 'OAuth':
            display(oauth_params)
        elif selection == 'Kerberos':
            display(kerberos_params)
        elif selection == 'TLS':
            ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
            ssl_context.check_hostname = False
            ssl_context.verify_mode = ssl.CERT_NONE
            display(tls_params,output_tls)
            ssl_context

# Call function when dropdown value changes
dropdown.observe(lambda change: display_widget_advanced_connection(change['new']), names='value')
logging.info('[Connection Page] Advanced setting connection done')

advanced_options = widgets.VBox([dropdown,output_advanced_connection, autocommit])

accordion = widgets.Accordion(children=[advanced_options], titles=('Advanced Options',))

In [ ]:
button = widgets.Button(description="Create", layout = {"align_self" : "center", "margin": "15px"})
button.style.button_color = button_color
button.style.text_color = text_color
output = widgets.Output(layout={'border': '1px solid lightgrey', 'width': output_width})
left_heading = widgets.HTML("""<h1 style="text-align: left;">Create a new connection</h1>""")
connection_and_accordian_vbox = widgets.VBox([connection_inputs, accordion], layout = {"height" : top_box_height})
left_section = widgets.VBox([left_heading, connection_and_accordian_vbox, button, output])

def on_button_clicked(b):
    output.clear_output(wait=True)  # Clear the output
    global ssl_context
    if dropdown.value =="TLS" and (tls_params=="Verify-Full" or tls_params=="Verify-CA"):
        ssl_context.load_verify_locations(cafile=uploader.value) # CA certificate used to verify server certificate
    with output:
        conn_info = {
           'host': host.value if host.value else "verticadb", 
           'port': port.value if port.value else 5433,
           'user': user.value if user.value else "dbadmin",
           'password': password.value if password.value else "",
           'database': database.value if database.value else "demo",
           'ssl': ssl_context if dropdown.value =="TLS" else False,
           'connection_timeout': eval(connection_timeout.value) if not connection_timeout.value == "" else 5,
        }
        logging.info(f"""[Connection Page] Trying to connect with the following parameters: {conn_info},""")
        vp.new_connection(
           conn_info,
           name = name.value if name.value else "VerticaDSN",
           auto = True,
           overwrite = True,)
        print("Successfully connected to the database:", vp.current_connection().options['database'])
button.on_click(on_button_clicked)

In [ ]:
from verticapy.connection import change_auto_connection

selection = widgets.Dropdown(
    options=vp.available_connections(),
    description='Available connections:',
    disabled=False, layout=widgets.Layout(width='300px'))
selection.style.description_width = '150px'
selectionvbox = widgets.VBox([selection], layout = {"height" : top_box_height})
button_select = widgets.Button(description="Reconnect", layout = {"align_self" : "center", "margin": "15px"})
button_select.style = button.style
output_2 = widgets.Output(layout={'border': '1px solid lightgrey', 'width': output_width})

right_heading = widgets.HTML("""<h1 style="text-align: left;">Select from existing connections</h1>""")
right_section = widgets.VBox([right_heading, selectionvbox,button_select, output_2])

def on_button_clicked_select(b):
    output_2.clear_output(wait=True)  # Clear the output
    with output_2:
        vp.connect(selection.value)
        change_auto_connection(selection.value)
        logging.info(f"""[Connection Page] Trying to connect to the following connection: {selection.value},""")
        print("Successfully connected to the database:", vp.current_connection().options['database'])
button_select.on_click(on_button_clicked_select)

In [ ]:
# Vertical Border line
vertica_border_line = widgets.HTML("""
<style>
    .vertical-line {
        border-left: 2px solid lightgrey; /* Adjust thickness and color as needed */
        height: 400px; /* Adjust height as needed */
    }
</style>
</head>
<body>

<div class="vertical-line"></div>

</body>
""")

In [ ]:
main_section = widgets.HBox([left_section, vertica_border_line, right_section], layout={"justify_content": "space-between", "margin" :"0px 50px 0px 50px"})


In [ ]:
from verticapy._utils._sql._vertica_version import vertica_version

button_version = widgets.Button(description="Click to refresh")
output_version = widgets.Output()

def on_button_clicked_version(b):
    output_version.clear_output(wait=True)  # Clear the output
    with output_version:
        print("Vertica version:", vertica_version()[0], ".", vertica_version()[1], ".", vertica_version()[2], ".", vertica_version()[3])
        print("VerticaPy version:", vp.__version__)
        print("VerticaPy long version:", vp.__long_version__)
button_version.on_click(on_button_clicked_version)
button_version.style = button.style

In [ ]:
import subprocess
output_install = widgets.Output()

def install_verticapy(_):
    with output_install:
        logging.info(f"""[Connection Page] Trying install verticapy""")
        subprocess.run(["pip", "install", "git+https://github.com/vertica/VerticaPy.git@master"])
        print("Successfully installed latest verticapy from the master branch.")
        print("Note: You must refresh to get the latest version.")
        logging.info(f"""Successfully isntalled verticapy""")

install_button = widgets.Button(description="Install latest verticapy")
install_button.on_click(install_verticapy)
install_button.style = button.style
install_verticapy_text = widgets.HTML("<br><b> Note: </b> If you want to get the latest VerticaPy then you can get the latest version directly from the GitHub repo by clicking below.")

In [ ]:
version_details = widgets.VBox([button_version, output_version, install_verticapy_text,install_button, output_install])
accordion = widgets.Accordion(children=[version_details], titles=('Version Details',))


In [ ]:
display(widgets.VBox([main_section,widgets.HTML("<br>"),accordion]))